In [2]:
%conda install openjdk -y
%pip install pyspark==3.3.0
%pip install spark-nlp==5.1.3
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.3.1
  latest version: 23.10.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.10.0



## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - openjdk


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2023.08.22 |       h06a4308_0         123 KB
    certifi-2023.11.17         |  py310h06a4308_0         158 KB
    openjdk-11.0.13            |       h87a67e3_0       341.0 MB
    ------------------------------------------------------------
                                           Total:       341.3 MB

The following NEW packages will be INSTALLED:

  openjdk            pkgs/main/linux-64::openjdk-11.0.13-h87a67e3_0 



In [3]:
import json
import time
import sagemaker
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import col, udf, explode
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from pyspark.sql.types import ArrayType, StringType
import sparknlp
from sparknlp.base import *
from pyspark.ml import Pipeline
from sparknlp.annotator import *
from sparknlp.pretrained import PretrainedPipeline

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [4]:
spark = SparkSession.builder \
    .appName("Spark NLP")\
    .master("local[*]")\
    .config("spark.driver.memory","16G")\
    .config("spark.driver.maxResultSize", "0") \
    .config("spark.kryoserializer.buffer.max", "2000M")\
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:5.1.3,org.apache.hadoop:hadoop-aws:3.2.2")\
    .config("fs.s3a.aws.credentials.provider","com.amazonaws.auth.ContainerCredentialsProvider")\
    .getOrCreate()

:: loading settings :: url = jar:file:/opt/conda/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-149699d0-7b03-42e4-b595-827bc4dd09ac;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;5.1.3 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.828 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.projectlombok#lombok;1.16.8 in central
	found com.google.cloud#google-cloud-storage;2.20.1 in central
	found com.google.guava#guava;31.1-jre in c

23/11/20 19:25:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [59]:
submissions = spark.read.parquet("s3a://dsan-6000-group-35/submissions.parquet", header=True)
comments = spark.read.parquet("s3a://dsan-6000-group-35/comments.parquet", header=True)

In [60]:
ts_sub = submissions.filter(col('text')
                           .rlike('(?i)' + '|'
                                  .join(['Taylor Swift', 'Taylor', 'Swift', 'Swizzle', 'Tay', 'Swiftie','Swifty','T-Swift','Miss Americana','Cruel Summer','All Too Well'])))
ts_com = comments.filter(col('body')
                           .rlike('(?i)' + '|'
                                  .join(['Taylor Swift', 'Taylor', 'Swift', 'Swizzle', 'Tay', 'Swiftie','Swifty','T-Swift','Miss Americana','Cruel Summer','All Too Well'])))

In [63]:
filtered_ts_com = ts_com.filter(~(col("author").like("%AutoModerator%") | 
                                  col("author").like("%SwiftBot13%") | 
                                  col("author").like("%songacronymbot%")))
combined_df = ts_sub.select("text").union(filtered_ts_com.select("body").withColumnRenamed("body", "text"))

In [70]:
from pyspark.ml import Pipeline
document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document") \
    .setCleanupMode("shrink")

sentenceDetector = SentenceDetector() \
.setInputCols(["document"]) \
.setOutputCol("sentence")

tokenizer = Tokenizer() \
  .setInputCols(["sentence"]) \
  .setOutputCol("token")

lemmatizer = LemmatizerModel.pretrained() \
.setInputCols(["token"]) \
.setOutputCol("lemma")

normalizer = Normalizer() \
    .setInputCols(["lemma"]) \
    .setOutputCol("normalized")

stopwords_cleaner = StopWordsCleaner()\
      .setInputCols("normalized")\
      .setOutputCol("cleanTokens")\
      .setCaseSensitive(False)

finisher = Finisher() \
    .setInputCols(["cleanTokens"]) \
    .setOutputCols(["tokens"]) \
    .setOutputAsArray(True) \
    .setCleanAnnotations(False)

nlp_pipeline = Pipeline() \
.setStages([
document_assembler,
sentenceDetector,
tokenizer,
lemmatizer,
normalizer,
stopwords_cleaner,
finisher
])

lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[OK!]


In [71]:
nlp_model = nlp_pipeline.fit(combined_df)
processed_df  = nlp_model.transform(combined_df)
tokens_df = processed_df.select('tokens').limit(10000)

In [72]:
from pyspark.ml.feature import CountVectorizer
cv = CountVectorizer(inputCol="tokens", outputCol="features", vocabSize=1000, minDF=3.0)
cv_model = cv.fit(tokens_df)
vectorized_tokens = cv_model.transform(tokens_df)

In [74]:
from pyspark.ml.clustering import LDA

num_topics = 10
lda = LDA(k=num_topics, maxIter=10)
model = lda.fit(vectorized_tokens)

In [75]:
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType

vocab = cv_model.vocabulary
topics = model.describeTopics()
topics_rdd = topics.rdd
topics_words = topics_rdd \
       .map(lambda row: row['termIndices']) \
       .map(lambda idx_list: [vocab[idx] for idx in idx_list]) \
       .collect()

def get_words(idx_list):
    return [vocab[idx] for idx in idx_list]

udf_get_words = udf(get_words, ArrayType(StringType()))
topics = topics.withColumn("words", udf_get_words(topics.termIndices))

topics_df = topics.select("topic", "words")

topics_df.show(truncate=False)

+-----+----------------------------------------------------------------------------------+
|topic|words                                                                             |
+-----+----------------------------------------------------------------------------------+
|0    |[Tay, concert, day, still, excited, album, music, listen, get, less]              |
|1    |[song, New, Taylor, Lover, Well, vote, Summer, like, Cruel, Wildest]              |
|2    |[version, Taylors, post, stream, taylor, Instagram, thread, week, album, Swifties]|
|3    |[album, track, amp, song, Red, release, autowebpamp, Fearless, TV, Taylors]       |
|4    |[Taylor, Swift, album, Swifts, song, ft, new, year, Album, make]                  |
|5    |[song, like, think, Taylor, album, get, love, know, make, Im]                     |
|6    |[Taylors, Version, amp, Vault, feat, vote, song, Stay, Red, round]                |
|7    |[piano, Piano, remix, link, version, gt, release, make, new, taylor]              |

In [76]:
topics_df.toPandas().to_csv('../../data/csv/nlp/ts_topic_model.csv',index=False)

To explore the popular discussions about Taylor Swift on Reddit, we adopted a comprehensive approach and extracted texts from Reddit submissions and comments containing various Taylor Swift-related keywords. We filtered out posts from automated accounts like `songacronymbot`, `AutoModerator` and `SwiftBot13` to focus on authentic human contributions. After merging these submissions and comments into a unified dataset, we processed the text through an NLP pipeline, generating about 10,000 unique tokens and vectorizing these for machine learning analysis. We then implemented Latent Dirichlet Allocation (LDA) for topic modeling, identifying prevalent themes and narratives within the Taylor Swift Reddit community. 

# df = pd.read_csv("../../data/csv/nlp/ts_topic_model.csv")

The analysis yielded ten distinct topics, reflecting the diverse interests of her fan community. Topics range from concert experiences and album anticipation (Topic 0) to in-depth discussions of specific songs like "Lover" and "Cruel Summer" (Topic 1). The community's engagement with Taylor's social media and streaming content is evident in Topic 2, while Topics 3 and 4 highlight conversations about her albums 'Red' and 'Fearless', and new music releases. Emotional responses and personal resonances with her music are captured in Topic 5, and there's keen interest in the creative variations of her songs, as seen in discussions about different versions and remixes (Topics 6 and 7). Analytical discussions on song lyrics and broader perspectives on her work (Topics 8 and 9) showcase the thoughtful and deep engagement of her fans. These topics illustrate the multifaceted nature of Taylor Swift's impact on the Reddit community.